<H1> Airbnb Review Analytics </H1>

Text analytics over the airbnb listings over the city of NYC.





# Imports and Loading the data

In [16]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
#import geopandas as gpd


In [21]:
#import geopandas as gpd

# Load the CSV data into pandas DataFrames
listings_df = pd.read_csv('../data/listings.csv')
neighbourhoods_df = pd.read_csv('../data/neighbourhoods.csv')
reviews_df = pd.read_csv('../data/reviews.csv')

In [22]:
reviews_df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r<br/>Nous avons ...
1,2595,19176,2009-12-05,53267,Cate,Great experience.
2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...
3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en..."
4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...
...,...,...,...,...,...,...
969481,1295054021439479562,1318207836791174950,2024-12-23,485243180,Vennela,The place was amazing! Great location and area...
969482,1295058207773420592,1302324242336666111,2024-12-01,368726917,Veronica,This was the perfect stay for our trip. Margar...
969483,1295058207773420592,1305143295960134335,2024-12-05,431518239,Rudy,….lugar increíble me sentí como en casa
969484,1295058207773420592,1309605174750125344,2024-12-11,26420925,Tara,We loved this place and would stay again. Our...


In [23]:
listings_df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,39572,https://www.airbnb.com/rooms/39572,20250103033441,2025-01-03,city scrape,1 br in a 2 br apt (Midtown West),NaN,NaN,https://a0.muscache.com/pictures/fd1bffd9-ccf8...,169927,...,5.00,4.98,4.86,NaN,f,2,1,1,0,0.25
1,39593,https://www.airbnb.com/rooms/39593,20250103033441,2025-01-03,city scrape,A lovely room w/ a Manhattan view,"A private, furnished large room to rent Jan/F...","Nate Silver called this super safe, clean, qui...",https://a0.muscache.com/pictures/0b9110f7-3b24...,110506,...,4.96,4.79,4.93,NaN,f,1,0,1,0,0.20
2,39704,https://www.airbnb.com/rooms/39704,20250103033441,2025-01-03,previous scrape,"Private, Large & Sunny 1BR w/W&D",It's a No Brainer:<br />•Terrific Space For Le...,The Neighborhood<br />• Rich History <br />• B...,https://a0.muscache.com/pictures/0bc4e8a4-c047...,170510,...,4.92,4.38,4.72,NaN,f,2,2,0,0,1.93
3,42300,https://www.airbnb.com/rooms/42300,20250103033441,2025-01-03,city scrape,Beautiful Lower East Side Loft,Architect-owned loft is a corner unit in a bea...,"The apartment is in the border of Soho, LES an...",https://a0.muscache.com/pictures/0e285e13-ee14...,184755,...,4.87,4.57,4.62,NaN,f,1,1,0,0,0.40
4,42729,https://www.airbnb.com/rooms/42729,20250103033441,2025-01-03,city scrape,@HouseOnHenrySt - Private 2nd bedroom w/shared...,NaN,"Lovely old Brooklyn neighborhood, with brick/b...",https://a0.muscache.com/pictures/925fe213-f5e1...,11481,...,4.73,4.58,4.64,NaN,f,4,1,3,0,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37779,1322024128476576885,https://www.airbnb.com/rooms/1322024128476576885,20250103033441,2025-01-03,city scrape,"The Gem of Riverdale, NYC!",Visiting NYC and looking for a place to stay? ...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,35391290,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
37780,1322305006441883773,https://www.airbnb.com/rooms/1322305006441883773,20250103033441,2025-01-03,city scrape,Amazing 1BR & 1BTH,"This special place is close to everything, mak...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,504878421,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
37781,1322523395081457963,https://www.airbnb.com/rooms/1322523395081457963,20250103033441,2025-01-03,city scrape,Home Share w/ Young Professionals - 2/3/4/5 Tr...,"Sun-filled, fully furnished apartment located ...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,2822805,...,NaN,NaN,NaN,NaN,f,21,0,21,0,NaN
37782,1322561224337225530,https://www.airbnb.com/rooms/1322561224337225530,20250103033441,2025-01-03,city scrape,Bedroom for rent in two bedroom UWS apartment,bedroom available for short and long-term rent...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,666385153,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN


In [24]:
listings_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [31]:
neighbourhoods_df

,neighbourhood_group,neighbourhood
0,Bronx,Allerton
1,Bronx,Baychester
2,Bronx,Belmont
3,Bronx,Bronxdale
4,Bronx,Castle Hill
...,...,...
225,Staten Island,Tottenville
226,Staten Island,West Brighton
227,Staten Island,Westerleigh
228,Staten Island,Willowbrook


In [33]:
neighbourhoods_df.columns

Index(['neighbourhood_group', 'neighbourhood'], dtype='object')

Number of Columns for every file : 

In [36]:
#Number of columns in listings_df

print("Listings : ", listings_df.columns.size)
print("Reviews : ", reviews_df.columns.size)
print("Neighborhoods: ", neighbourhoods_df.columns.size)

Listings :  75
Reviews :  6
Neighborhoods:  2


Shape for every file: 

In [37]:
print("Listings : ", listings_df.shape)
print("Reviews : ", reviews_df.shape)
print("Neighborhoods: ", neighbourhoods_df.shape)

Listings :  (37784, 75)
Reviews :  (969486, 6)
Neighborhoods:  (230, 2)


# Exploratory Data Analysis

# Preprocessing the Text

# Feature Engineering